In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from random import randint

In [2]:
def norm(C) :
    tmp = 0
    for i in range(len(C)) :
        tmp += C[i]*C[i]
        
    return sqrt(tmp)

In [3]:
def dist(Cp,Cc) :
    norm1 = norm(Cp)
    norm2 = norm(Cc)
    tmp = 0
    if (len(Cp) == 0) | (len(Cc) == 0) :
        return 0
    else :
        for i in range(min(len(Cp),len(Cc))) :
            tmp += Cp[i] * Cc[i]
        return tmp/(norm1*norm2)

In [4]:
def init(nblog,countlog) :
    trace = pd.read_csv(nblog, header=None)
    trace.columns = ['syscall']
    index = pd.read_csv(countlog, index_col=0)
    index = index.drop('occur',1)
    index.reset_index(inplace=True)
    index.set_index('name', inplace=True)
    index.columns = ['ind']
    nmdb = pd.DataFrame(np.empty((0,3)))
    nmdb.columns = ['key','value','num']
    nmdb.set_index('key', inplace=True)
    S = 5000 # Epoch size
    Tt = 0.99 # Training threshold
    Bs = 10 # Bag size
    return trace, index, nmdb, S, Tt, Bs

In [5]:
def train(nblog,countlog) :
    trace, index, nmdb, S, Tt, Bs = init(nblog,countlog)
    Cp = []
    Cc = []
    key = [0] * len(index)
    ep = 0
    while (dist(Cp,Cc)  < Tt) | (ep*S < 0.05*len(trace)) :
        Cp = Cc.copy()
        Cc = [0]*len(Cc)
        rp = ep
        #rp = randint(0,int(len(trace)/S) - 1) # comment to choose consecutive epochs, need to generate unique
        for i in range((S - Bs + 1)) :
            for k in range(len(key)) :
                key[k] = 0
            for j in range(Bs) :
                if str(trace.loc[rp*S+i+j].syscall) in index.index :    
                    ind = int(index.loc[str(trace.loc[rp*S+i+j].syscall)].ind)
                else :
                    ind = int(index.loc['other'].ind)
                key[ind] += 1
            if str(key) in nmdb.index :
                nmdb.at[str(key), 'value'] += 1
                num = int(nmdb.loc[str(key)].num)
                Cc[num] += 1
            else :
                b = pd.DataFrame(np.empty((0,3)))
                b.columns = ['key','value','num']
                b = b.append({'key': str(key), 'value' : 1, 'num' : len(nmdb)}, ignore_index = True)
                b.set_index('key', inplace=True)
                nmdb = nmdb.append(b)
                num = int(nmdb.loc[str(key)].num)
                Cc.insert(num,1)
        ep += 1
    return nmdb, index, S, Bs, ep

In [6]:
#nmdb, _, _, _, ep = train('logGOODparsed.txt','count.csv')
#print('There are %d epochs' % (ep))

There are 12 epochs
